# Scrapy

Le web scraping (parfois appelé harvesting) est une technique d'extraction du contenu de sites Web, via un script
ou un programme, dans le but de le transformer pour permettre son utilisation dans un autre contexte, par exemple
le référencement.


Le web scraping peut être utilisé pour récupérer des métadonnées. C'est ainsi que la bibliothèque multimédia 
XBMC récupère les informations sur les médias qu'elle gère (affiches et résumés des films et séries télévisées, 
jaquette des albums…) sur différents sites tels que IMDb ou AlloCiné au moyen de scrapers dédiés.


Il y plusieurs types de librairie | framework qui nous permettent de faire du WebScraping. 
Notamment Scrapy, Selenium, BeautifulSoup, pour ne citer que les plus connus.

Scrapy est un framework open-source permettant la création de robots d'indexation. Développé en Python, il 
dispose d'une forte communauté, offrant de nombreux modules supplémentaires. La première version stable a été 
publiée en septembre 2009. Depuis, l'équipe de développement publie régulièrement de nouvelles versions dans le 
but d'enrichir le framework en fonctionnalité. L'objectif principal est d'obtenir une API stable pour la version
1.02. Le framework dispose d'une communauté active, et un support commercial est effectué par plusieurs entreprises.

Le framework est compatible Python 2.7 et Python 3.6 ou au-dessus, sur la majorité des plates-formes.


**Caractéristiques de Scrapy**

L'équipe responsable du développement du framework lui confère plusieurs caractéristiques :

    1. Simple : aucune notion avancée en Python n'est nécessaire pour utiliser Scrapy
    2. Productif : l'empreinte de code à générer est très courte, la plupart des opérations sont gérées par Scrapy
    3. Rapide : le framework est rapide, avec une gestion d'actions en parallèle notamment
    4. Extensible : chaque robot peut être personnalisés via des extensions, modifiant son comportement
    5. Portable : les robots Scrapy sont compatibles Linux, Windows, Mac et BSD
    6. Open Source
    7. Robuste, grâce à une batterie de tests effectuées aussi bien par les développeurs que la communauté



Pour installer scrapy : pip install scrapy



Avec conda : conda install -c conda-forge scrapy

Documentation de scrapy :

https://docs.scrapy.org/en/latest/index.html

Pour connaître les différentes étapes de la création d'un jeu de données avec scrapy.

https://ledatascientist.com/creer-un-jeu-de-donnees-avec-scrapy/

## Exemple de Robot:

Le site web du projet propose un tutoriel détaillé sur l'utilisation de Scrapy5. Ce cours propose notamment 
plusieurs exemples, comme celui ci-contre, extrayant certains liens présents sur plusieurs pages web.



In [ ]:
from scrapy.spider import BaseSpider
from scrapy.selector import HtmlXPathSelector

from tutorial.items import DmozItem

class DmozSpider(BaseSpider):
   name = "dmoz"
   allowed_domains = ["dmoz.org"]
   start_urls = [
       "http://www.dmoz.org/Computers/Programming/Languages/Python/Books/",
       "http://www.dmoz.org/Computers/Programming/Languages/Python/Resources/"
   ]

   def parse(self, response):
       hxs = HtmlXPathSelector(response)
       sites = hxs.select('//ul/li')
       items = []
       for site in sites:
           item = DmozItem()
           item['title'] = site.select('a/text()').extract()
           item['link'] = site.select('a/@href').extract()
           item['desc'] = site.select('text()').extract()
           items.append(item)
       return items



### Finalisation du Spider

La dernière étape pour avoir de la donnée est de retourner des items. Cette étape est essentielle. 
Et elle consiste juste à créer une instance de ReviewsAllocineItem à chaque commentaire, à renseigner chacun de 
ses champs avec les valeurs scrapées puis les “yield”. Notre fichier spider final à ça :


In [3]:

from scrapy import Request, Spider
from ..items import ReviewsAllocineItem
 
class SpiderReviewsAllocine(Spider):
    # Nom du spider
    name = "reviews_allocine"
    # URL de la page à scraper
    url = "https://www.allocine.fr/film/fichefilm-10568/critiques/spectateurs/"
 
    def start_requests(self):
        yield Request(url=url, callback=self.parse_films)
 
    def parse_films(self, response):
        title = response.css("a.titlebar-link::text").extract_first()
        review_blocks = response.css("div.review-card")
        for review_card in review_blocks:
            review = review_card.css("div.content-txt::text").extract_first()
            stars = review_card.css("span.stareval-note::text").extract_first()
             
            # Pour avoir la note (nombre d'étoiles) en float
            stars = float(stars.replace(",", "."))
 
            item = ReviewsAllocineItem()
 
            item['title'] = title
            item['stars'] = stars
            item['review'] = review
 
            yield item



SyntaxError: invalid syntax (<ipython-input-3-9ae605ceddd8>, line 31)

Pour lancer notre spider et avoir les données scrapées dans un fichier CSV, on fait la commande suivante :
scrapy crawl reviews_allocine -o reviews_allocine.csv

### Sources utiles pour la création d'un jeu de données avec scrapy

https://medium.com/@alexandrewrg/scraper-les-donn%C3%A9es-de-plusieurs-pages-avec-scrapy-2e076ac7dc09


https://ledatascientist.com/creer-un-jeu-de-donnees-avec-scrapy/
